# Test Tools and Models

## Running this notebook

To run this in VS Code:
1. Open a terminal
2. Navigate to `./backend`
3. Configure poetry to set up a venv environment by running `poetry config virtualenvs.in-project true`
4. Remove any existing poetry venv by running `poetry remove --all`
5. Run `poetry install` to install the dependencies in the virtual environment
6. In Select Kernel dropdown in the top right corner of VS Code, select `Python Environments` and select the venv environment you created at `backend/.venv/bin/python`
7. Run the notebook by clicking on the play button in the top right corner of VS Code or running `jupyter notebook test_notebook.ipynb`

In [ ]:
import sys
import os
import json
import logging

# Get the absolute path to the src directory
src_path = os.path.abspath('src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from dotenv import load_dotenv
load_dotenv('src/.env')
os.environ['MILVUS_HOST'] = 'localhost'
os.environ['OLLAMA_HOST'] = 'localhost'
from src.models import get_model, bind_tools, handle_tool_call, handle_streaming_tool_call
from src.tools import retrieve_context, multiply
from src.langgraph_agent import create_agent_graph, run_agent_graph, run_agent_graph_streaming
from src.app import ChatRequest, prompt_templates

In [ ]:
# test_notebook.ipynb (Revised)

assistant_system_prompt = prompt_templates["assistant_system_prompt"]
tools = [multiply, retrieve_context]
agent_graph = create_agent_graph(tools, assistant_system_prompt)
user_prompt = "What is the phone number for DataNinja Support?"

async def stream_generator(agent_graph, user_prompt):
    try:
        async for chunk in run_agent_graph_streaming(agent_graph, user_prompt):
            if chunk:
                yield f"data: {json.dumps({'content': chunk})}\n\n"
    except Exception as e:
        yield f"data: {json.dumps({'error': str(e)})}\n\n"
    finally:
        yield "data: [DONE]\n\n"

async def test_agent_graph_streaming():
    async for chunk in stream_generator(agent_graph, user_prompt):
        print(chunk)

# Execute the test in Jupyter:
await test_agent_graph_streaming()  # Use await in a Jupyter cell or use %async magic